# Explore kalman filter



This notebook explores how to run the Kalman filter on mock data 

In [1]:
%load_ext autoreload
%autoreload 2

# 1. Get the data to pass to the filter 

In [2]:
import sys
import glob 
sys.path.append('../src')
from data_loader import LoadWidebandPulsarData

In [3]:

import pandas as pd 

datadir = '../data/IPTA_MockDataChallenge/IPTA_Challenge1_open/Challenge_Data/Dataset2/'


# Get all .par files in the directory
par_files = sorted(glob.glob(datadir + '*.par'))
tim_files = sorted(glob.glob(datadir + '*.tim'))

assert len(par_files) == len(tim_files) 


# Combine par_files and tim_files into pairs
file_pairs = list(zip(par_files, tim_files))



dfs = []
dfs_meta = []
total_num_rows = 0 
i = 0
# Check we can load the files with no errors
for par_file, tim_file in file_pairs[0:2]:
    psr = LoadWidebandPulsarData.read_par_tim(par_file, tim_file)

    print(psr.M_matrix.shape)
    print(psr.fitpars)

    df = pd.DataFrame({'toas': psr.toas, f'residuals_{i}': psr.residuals})

    df_meta = pd.DataFrame({'name': [psr.name], f'dim_M': [psr.M_matrix.shape[-1]],f'RA': [psr.RA],f'DEC': [psr.DEC]})

    dfs.append(df)
    dfs_meta.append(df_meta)

    total_num_rows += len(df)
    i += 1
 






[tempo2Util.C:396] Warning: [TIM1] Please place MODE flags in the parameter file 
(130, 8)
['Offset', 'RAJ', 'DECJ', 'F0', 'F1', 'PMRA', 'PMDEC', 'PX']
[tempo2Util.C:401] Warning: [DUP1] duplicated warnings have been suppressed.


(130, 11)
['Offset', 'RAJ', 'DECJ', 'F0', 'F1', 'PX', 'PB', 'A1', 'TASC', 'EPS1', 'EPS2']


In [4]:
dfs_meta 

[         name  dim_M        RA       DEC
 0  J0030+0451      8  0.132895  0.084841,
          name  dim_M      RA       DEC
 0  J0218+4232     11  0.6026  0.742431]

In [5]:
from functools import reduce 

merged_df = reduce(lambda left, right: pd.merge(left, right, on='toas', how='outer'), dfs)


In [6]:
merged_df 

,toas,residuals_0,residuals_1
0,4.580106e+09,3.380995e-06,NaN
1,4.580106e+09,NaN,-9.349598e-07
2,4.581370e+09,2.576195e-06,NaN
3,4.581370e+09,NaN,7.212309e-06
4,4.582755e+09,1.867198e-06,NaN
...,...,...,...
255,4.731184e+09,-9.911458e-07,NaN
256,4.732025e+09,NaN,-1.214947e-05
257,4.732025e+09,-8.987281e-07,NaN
258,4.733030e+09,-1.237823e-06,NaN


In [7]:
combined_df = pd.concat(dfs_meta, ignore_index=True)


In [8]:
combined_df

,name,dim_M,RA,DEC
0,J0030+0451,8,0.132895,0.084841
1,J0218+4232,11,0.602600,0.742431


# 2. Specify the model to use with the filter 

In [9]:
from models import StochasticGWBackgroundModel

In [10]:
psr_information = combined_df

In [11]:
combined_df

,name,dim_M,RA,DEC
0,J0030+0451,8,0.132895,0.084841
1,J0218+4232,11,0.602600,0.742431


In [12]:
model = StochasticGWBackgroundModel(psr_information)

In [17]:

θ = {'dt': 0.50}
F_array = model.F_matrix(θ)
Q_array = model.Q_matrix(θ)

TypeError: 'numpy.ndarray' object is not callable

In [14]:
F_array

array([[1.   , 0.5  , 0.125, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 1.   , 0.5  , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   ,

# 3. Now run the filter

In [15]:
from kalman_filter import KalmanFilter

In [16]:


observations = merged_df 
model = 


x0 = 
P0 = #maybe should be part of the model?
self, model, observations, x0, P0

SyntaxError: invalid syntax (3739317549.py, line 2)